In [ ]:
!pip install transformers datasets evaluate accelerate
!pip install sentencepiece
!pip install huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset

dataset = load_dataset("xlangai/spider")
print(dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.51k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/831k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/126k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1034 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 1034
    })
})


Imports and Initialization, Normalization Function ,Preprocessing Function, Dataset Processing

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer
import re

# Load tokenizer and dataset
tokenizer = AutoTokenizer.from_pretrained("t5-large")
dataset = load_dataset("b-mc2/sql-create-context")

def normalize(text):
    return re.sub(r"\s+", " ", text.strip().lower())

# Construct input with question + context (table schema)
def preprocess(example):
    context = normalize(example["context"])
    question = normalize(example["question"])
    query = normalize(example["answer"])

    input_text = f"translate to SQL: {question} [SEP] {context}"
    input_ids = tokenizer(input_text, padding="max_length", truncation=True, max_length=512)
    target_ids = tokenizer(query, padding="max_length", truncation=True, max_length=128)

    return {
        "input_ids": input_ids["input_ids"],
        "attention_mask": input_ids["attention_mask"],
        "labels": target_ids["input_ids"]
    }

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset["train"].column_names, batched=False)


README.md:   0%|          | 0.00/4.43k [00:00<?, ?B/s]

sql_create_context_v4.json:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/78577 [00:00<?, ? examples/s]

Map:   0%|          | 0/78577 [00:00<?, ? examples/s]

In [ ]:
!pip install datasets transformers wandb


importing necessary Python libraries and initializing a tokenizer for the T5-large model.

In [ ]:
import re
import json
import os
from datasets import load_dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-large")


In [ ]:
# Load the Spider dataset from Hugging Face (or another dataset if needed)
dataset = load_dataset("xlangai/spider")

# Check the dataset structure
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 7000
    })
    validation: Dataset({
        features: ['db_id', 'query', 'question', 'query_toks', 'query_toks_no_value', 'question_toks'],
        num_rows: 1034
    })
})


Loading and processing a database schema file.

In [ ]:
import json

# Load schema from the uploaded file
schema_path = "/tables.json"  # Path to the uploaded tables.json file

with open(schema_path, "r") as f:
    spider_schemas = json.load(f)

# Build a mapping from db_id to schema
schema_map = {schema["db_id"]: schema for schema in spider_schemas}


In [ ]:
import re
import json
import urllib.request
from datasets import load_dataset
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("t5-large")


In [ ]:
# Load the Spider dataset from Hugging Face
dataset = load_dataset("xlangai/spider")

# Load schema from the uploaded tables.json file
schema_path = "/tables.json"  # Path to the uploaded tables.json file
with open(schema_path, "r") as f:
    spider_schemas = json.load(f)

# Build a mapping from db_id to schema
schema_map = {schema["db_id"]: schema for schema in spider_schemas}


Defining utility functions to normalize and format database schema information

In [ ]:
# Utils to normalize schema text
def normalize_column_name(name):
    return re.sub(r"[^a-zA-Z0-9_]", "_", name.strip().lower())

def construct_schema(schema):
    tables = [normalize_column_name(name) for name in schema["table_names_original"]]
    columns = []
    for table_id, column_name in schema["column_names_original"]:
        if table_id >= 0:
            table_name = tables[table_id]
            columns.append(f"{normalize_column_name(column_name)} ({table_name})")
    return " | ".join(tables + columns)


Defining a preprocessing function for training a sequence-to-sequence model

In [ ]:
# Preprocessing function
def preprocess(example):
    db_id = example["db_id"]
    if db_id not in schema_map:
        return None  # skip if schema not found

    schema_str = construct_schema(schema_map[db_id])
    question = example["question"]
    query = example["query"]

    input_text = f"translate to SQL: {question} [SEP] {schema_str}"
    input_ids = tokenizer(input_text, padding="max_length", truncation=True, max_length=256)  # Reduced max length
    target_ids = tokenizer(query, padding="max_length", truncation=True, max_length=64)  # Reduced max length

    return {
        "input_ids": input_ids["input_ids"],
        "attention_mask": input_ids["attention_mask"],
        "labels": target_ids["input_ids"]
    }

# Apply preprocessing to the dataset
tokenized_dataset = dataset.map(preprocess, remove_columns=dataset["train"].column_names, batched=False)


Map:   0%|          | 0/7000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

Splitting the Data into Train and Validation Sets

In [ ]:
# Split the train dataset into train and validation sets (80% train, 20% validation)
train_size = 0.8
train_dataset = tokenized_dataset["train"].train_test_split(test_size=1-train_size)

# Now, you can use train_dataset['train'] and train_dataset['test'] as your datasets


Training Arguments

In [ ]:
# Set up training arguments (adjust as needed)
training_args = TrainingArguments(
    output_dir="./results",                # where to save the model
    eval_steps=500,                         # evaluation every 500 steps (you can adjust this)
    learning_rate=2e-5,                     # learning rate
    per_device_train_batch_size=4,          # batch size for training
    per_device_eval_batch_size=4,           # batch size for evaluation
    num_train_epochs=3,                     # number of epochs
    weight_decay=0.01,                      # regularization
    logging_dir="./logs",                   # where to save logs
    logging_steps=10,                       # how often to log
    save_steps=1000,                        # save model checkpoints after 1000 steps
)


initializes a Hugging Face Trainer to fine-tune a pre-trained model

In [ ]:
# Initialize the Trainer
trainer = Trainer(
    model=model,                             # our pre-trained model
    args=training_args,                      # the training arguments
    train_dataset=train_dataset['train'],    # tokenized training dataset
    eval_dataset=train_dataset['test'],      # tokenized validation dataset
)


In [ ]:
# Start training
trainer.train()


Step,Training Loss
10,5.904000
20,2.585100
30,1.802400
40,1.554600
50,1.580400
60,1.080700
70,1.052400
80,0.889500
90,0.907400
100,0.877800


Step,Training Loss
10,5.904000
20,2.585100
30,1.802400
40,1.554600
50,1.580400
60,1.080700
70,1.052400
80,0.889500
90,0.907400
100,0.877800


TrainOutput(global_step=4200, training_loss=0.2744468288478397, metrics={'train_runtime': 5858.0836, 'train_samples_per_second': 2.868, 'train_steps_per_second': 0.717, 'total_flos': 1.81863972864e+16, 'train_loss': 0.2744468288478397, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./fine-tuned-t5-sql")
tokenizer.save_pretrained("./fine-tuned-t5-sql")


('./fine-tuned-t5-sql/tokenizer_config.json',
 './fine-tuned-t5-sql/special_tokens_map.json',
 './fine-tuned-t5-sql/spiece.model',
 './fine-tuned-t5-sql/added_tokens.json',
 './fine-tuned-t5-sql/tokenizer.json')

Evaluate on Validation

In [ ]:
import json

with open("/tables.json", "r") as f:
    spider_schemas = json.load(f)

# Build the schema map
schema_map = {schema["db_id"]: schema for schema in spider_schemas}


In [ ]:
import re

def normalize_column_name(name):
    return re.sub(r"[^a-zA-Z0-9_]", "_", name.strip().lower())

def construct_schema(schema):
    tables = [normalize_column_name(name) for name in schema["table_names_original"]]
    columns = []
    for table_id, column_name in schema["column_names_original"]:
        if table_id >= 0:
            table_name = tables[table_id]
            columns.append(f"{normalize_column_name(column_name)} ({table_name})")
    return " | ".join(tables + columns)


In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch

# Load model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./fine-tuned-t5-sql")
tokenizer = T5Tokenizer.from_pretrained("t5-large")
model.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=1024, out_features=4096, bias=False)
              (wo): Linear(in_features=4096, out_features=1024, bias=False)
              (d

In [ ]:
import json

with open("/tables.json", "r") as f:
    spider_schemas = json.load(f)

schema_map = {schema["db_id"]: schema for schema in spider_schemas}


In [ ]:
import re

def normalize_column_name(name):
    return re.sub(r"[^a-zA-Z0-9_]", "_", name.strip().lower())

def construct_schema(schema):
    tables = [normalize_column_name(name) for name in schema["table_names_original"]]
    columns = []
    for table_id, column_name in schema["column_names_original"]:
        if table_id >= 0:
            table_name = tables[table_id]
            columns.append(f"{normalize_column_name(column_name)} ({table_name})")
    return " | ".join(tables + columns)


In [ ]:
# Pick a few examples from the validation split
examples = train_dataset["test"][:5]


In [ ]:
question = "What is the average salary of employees in the department?"

input_text = f"translate to SQL: {question} [SEP] {schema_str}"
print(f"Input Text: {input_text}")

inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_length=100)

predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Predicted SQL: {predicted_sql}")


Input Text: translate to SQL: What is the average salary of employees in the department? [SEP] employees | salary (employees) | department (employees)
Predicted SQL: SELECT avg(T1.salary) FROM employees AS T1 JOIN department AS T2 ON T1.department = T2.department


Input Text: translate to SQL: What is the average salary of employees in the department? [SEP] employees | salary (employees) | department (employees)
Predicted SQL: SELECT avg(T1.salary) FROM employees AS T1 JOIN department AS T2 ON T1.department = T2.department


In [ ]:
question = "What is the name of the highest paid employee?"

# Manually crafting schema
schema_str = "employees | name (employees) | salary (employees)"
input_text = f"translate to SQL: {question} [SEP] {schema_str}"

inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

with torch.no_grad():
    outputs = model.generate(**inputs, max_length=100)

predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Predicted SQL: {predicted_sql}")


Predicted SQL: SELECT name FROM employees ORDER BY salary DESC LIMIT 1


In [ ]:
# Define 10 sample questions and schemas to test the model
test_data = [
    {
        "question": "What is the average salary of employees in the department?",
        "schema": "employees | salary (employees) | department (employees)"
    },
    {
        "question": "What is the name of the highest paid employee?",
        "schema": "employees | name (employees) | salary (employees)"
    },
    {
        "question": "Which employees work in the sales department?",
        "schema": "employees | name (employees) | department (employees)"
    },
    {
        "question": "How many employees are in each department?",
        "schema": "employees | department (employees)"
    },
    {
        "question": "What is the total salary paid to employees in each department?",
        "schema": "employees | salary (employees) | department (employees)"
    },
    {
        "question": "Which employee has the lowest salary?",
        "schema": "employees | name (employees) | salary (employees)"
    },
    {
        "question": "What is the name of the employee with the highest salary in the marketing department?",
        "schema": "employees | name (employees) | salary (employees) | department (employees)"
    },
    {
        "question": "Which employees have a salary greater than 50000?",
        "schema": "employees | name (employees) | salary (employees)"
    },
    {
        "question": "What is the average age of employees in the IT department?",
        "schema": "employees | age (employees) | department (employees)"
    },
    {
        "question": "Which employees joined after January 1, 2020?",
        "schema": "employees | name (employees) | join_date (employees)"
    }
]

# Test the model on these queries
for test_case in test_data:
    question = test_case["question"]
    schema_str = test_case["schema"]
    input_text = f"translate to SQL: {question} [SEP] {schema_str}"

    # Tokenize the input and run it through the model
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100)

    # Decode the output to get the SQL query
    predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the results
    print(f"Question: {question}")
    print(f"Input Text: {input_text}")
    print(f"Predicted SQL: {predicted_sql}")
    print("-" * 80)


Question: What is the average salary of employees in the department?
Input Text: translate to SQL: What is the average salary of employees in the department? [SEP] employees | salary (employees) | department (employees)
Predicted SQL: SELECT avg(T1.salary) FROM employees AS T1 JOIN department AS T2 ON T1.department = T2.department
--------------------------------------------------------------------------------
Question: What is the name of the highest paid employee?
Input Text: translate to SQL: What is the name of the highest paid employee? [SEP] employees | name (employees) | salary (employees)
Predicted SQL: SELECT name FROM employees ORDER BY salary DESC LIMIT 1
--------------------------------------------------------------------------------
Question: Which employees work in the sales department?
Input Text: translate to SQL: Which employees work in the sales department? [SEP] employees | name (employees) | department (employees)
Predicted SQL: SELECT T1.name FROM employees AS T1 J

Testing own or unseen data schema

In [ ]:
# Define test data based on unseen schema (Worker, Bonus, Title)
test_data_unseen = [
    {
        "question": "What is the total bonus given to each worker?",
        "schema": "Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus)"
    },
    {
        "question": "What is the title of the worker with WORKER_ID 001?",
        "schema": "Worker | WORKER_ID (Worker) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title)"
    },
    {
        "question": "Which workers received a bonus amount greater than 3000?",
        "schema": "Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus)"
    },
    {
        "question": "List the names of workers who are Managers.",
        "schema": "Worker | WORKER_ID (Worker) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title)"
    },
    {
        "question": "What is the total bonus amount given to workers with Manager title?",
        "schema": "Worker | WORKER_ID (Worker) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus)"
    },
    {
        "question": "Who are the workers that received a bonus in 2016?",
        "schema": "Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus) | BONUS_DATE (Bonus)"
    },
    {
        "question": "What is the average bonus given to employees in 2016?",
        "schema": "Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus) | BONUS_DATE (Bonus)"
    },
    {
        "question": "What is the job title of the worker with WORKER_ID 002?",
        "schema": "Worker | WORKER_ID (Worker) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title)"
    },
    {
        "question": "Which workers have received a bonus in both February and June of 2016?",
        "schema": "Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus) | BONUS_DATE (Bonus)"
    },
    {
        "question": "Which worker was given a bonus in February 2016?",
        "schema": "Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus) | BONUS_DATE (Bonus)"
    }
]

# Test the model on unseen data schema
for test_case in test_data_unseen:
    question = test_case["question"]
    schema_str = test_case["schema"]
    input_text = f"translate to SQL: {question} [SEP] {schema_str}"

    # Tokenize the input and run it through the model
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100)

    # Decode the output to get the SQL query
    predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the results
    print(f"Question: {question}")
    print(f"Input Text: {input_text}")
    print(f"Predicted SQL: {predicted_sql}")
    print("-" * 80)


Question: What is the total bonus given to each worker?
Input Text: translate to SQL: What is the total bonus given to each worker? [SEP] Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus)
Predicted SQL: SELECT sum(BONUS_AMOUNT) , WORKER_ID FROM Bonus GROUP BY WORKER_ID
--------------------------------------------------------------------------------
Question: What is the title of the worker with WORKER_ID 001?
Input Text: translate to SQL: What is the title of the worker with WORKER_ID 001? [SEP] Worker | WORKER_ID (Worker) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title)
Predicted SQL: SELECT WORKER_TITLE FROM Title WHERE WORKER_ID = 001
--------------------------------------------------------------------------------
Question: Which workers received a bonus amount greater than 3000?
Input Text: translate to SQL: Which workers received a bonus amount greater than 3000? [SEP] Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID

Multiple Queries with One Schema

In [ ]:
# Define complete schema once
complete_schema = """
Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title)
"""

# Define multiple questions
questions = [
    "What is the total bonus given to each worker?",
    "What is the title of the worker with WORKER_ID 001?",
    "Which workers received a bonus amount greater than 3000?",
    "List the names of workers who are Managers.",
    "What is the total bonus amount given to workers with Manager title?",
    "Who are the workers that received a bonus in 2016?",
    "What is the average bonus given to employees in 2016?",
    "What is the job title of the worker with WORKER_ID 002?",
    "Which workers have received a bonus in both February and June of 2016?",
    "Which worker was given a bonus in February 2016?"
]

# Test the model with the complete schema once
for question in questions:
    input_text = f"translate to SQL: {question} [SEP] {complete_schema}"

    # Tokenize the input and run it through the model
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=256).to(device)

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=100)

    # Decode the output to get the SQL query
    predicted_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Print the results
    print(f"Question: {question}")
    print(f"Input Text: {input_text}")
    print(f"Predicted SQL: {predicted_sql}")
    print("-" * 80)


Question: What is the total bonus given to each worker?
Input Text: translate to SQL: What is the total bonus given to each worker? [SEP] 
Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title)

Predicted SQL: SELECT sum(BONUS_AMOUNT) , WORKER_ID FROM Bonus GROUP BY WORKER_ID
--------------------------------------------------------------------------------
Question: What is the title of the worker with WORKER_ID 001?
Input Text: translate to SQL: What is the title of the worker with WORKER_ID 001? [SEP] 
Worker | WORKER_ID (Worker) | Bonus | BONUS_AMOUNT (Bonus) | WORKER_REF_ID (Bonus) | Title | WORKER_REF_ID (Title) | WORKER_TITLE (Title)

Predicted SQL: SELECT WORKER_TITLE FROM WORKER WHERE WORKER_ID = 001
--------------------------------------------------------------------------------
Question: Which workers received a bonus amount greater than 3000?
Input Text: translate to SQL: Which workers received

In [ ]:
from sklearn.metrics import accuracy_score

# List of ground truth (correct) SQL queries
ground_truth = [
    "SELECT sum(BONUS_AMOUNT) , WORKER_ID FROM Bonus GROUP BY WORKER_ID",
    "SELECT WORKER_TITLE FROM Title WHERE WORKER_ID = 001",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_AMOUNT > 3000",
    "SELECT WORKER_TITLE FROM Worker WHERE WORKER_ID = 'Manager'",
    "SELECT sum(BONUS_AMOUNT) FROM Bonus WHERE WORKER_TITLE = 'Manager'",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = '2016'",
    "SELECT avg(BONUS_AMOUNT) FROM Bonus WHERE BONUS_DATE = 2016",
    "SELECT WORKER_TITLE FROM Title WHERE WORKER_ID = 002",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = 'February' INTERSECT SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = 'June'",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = 'February 2016'"
]

# List of predicted SQL queries (generated by the model)
predicted_sql = [
    "SELECT sum(BONUS_AMOUNT) , WORKER_ID FROM Bonus GROUP BY WORKER_ID",
    "SELECT WORKER_TITLE FROM WORKER WHERE WORKER_ID = 001",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_AMOUNT > 3000",
    "SELECT WORKER_TITLE FROM Worker WHERE WORKER_ID = 'Manager'",
    "SELECT sum(BONUS_AMOUNT) FROM Bonus WHERE Title = 'Manager'",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = 2016",
    "SELECT avg(BONUS_AMOUNT) FROM Bonus WHERE BONUS_DATE = 2016",
    "SELECT WORKER_TITLE FROM Title WHERE WORKER_ID = 002",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = 'February' INTERSECT SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = 'June'",
    "SELECT WORKER_ID FROM Bonus WHERE BONUS_DATE = 'February 2016'"
]

# Define a function to calculate accuracy
def calculate_accuracy(ground_truth, predicted_sql):
    # Check for exact match between predicted and ground truth SQL queries
    correct = [1 if gt == pred else 0 for gt, pred in zip(ground_truth, predicted_sql)]

    # Calculate accuracy
    accuracy = sum(correct) / len(correct)
    return accuracy

# Calculate accuracy
accuracy = calculate_accuracy(ground_truth, predicted_sql)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 70.00%
